# TCC Augusto e Ícaro
## Modelo de automatização das Heurísticas de Nielsen para comentários em reviews de Apps

* Versão 1.0
* Bibliotecas utilizadas: pandas, numpy, texthero, ntlk e corpus do ntlk em português
* Dataset utilizado: dataset_v4.csv
* Data: 22/07/2020

### Objetivos:

[x] Pre processamento detalhado

[x] Pipeline de pre-processamento

[x] Classificador baseado em ocorrência

In [497]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install texthero 
!{sys.executable} -m pip install nltk 

In [498]:
import pandas as pd
import texthero as hero
import nltk
import numpy as np
# nltk.download()

## Pré-processamento

Nessa versão iremos testar apenas com textos em ingles, sem adicionar os pesos de sentimentos e heurísticas de Nielsen. Queremos treinar o modelo para classificar apenas em usabilidade ou não.

In [499]:
cols = ['Text', 'is_usability', 'is_classified']
df = pd.read_csv("reviews_v4.csv", index_col=False, usecols=cols)
df = df.rename(columns={'Text': 'text'})
df.head()

,text,is_usability,is_classified
0,Negócio de reconhecimento facialnão funciona. ...,True,True
1,Um lixo!!! 9 tentativas de econhecimento facia...,False,True
2,Horrível! Pior que FBI! Se fosse pra receberem...,True,True
3,Meio difícil fazer sem óculos mais deu certo,True,True
4,"Não serve pra nada , não dá para acessar,péssimo.",True,True


In [500]:
df['is_usability'].value_counts()

False    755
True     175
Name: is_usability, dtype: int64

In [501]:
df['is_classified'].value_counts()

False    548
True     381
Name: is_classified, dtype: int64

### Limpando dados

O pipeline padrão remove dígitos, pontuação, remove diacritics, stopwords em inglês e whitespace

In [502]:
pipeline = hero.preprocessing.get_default_pipeline()
pipeline

[<function texthero.preprocessing.fillna(input: pandas.core.series.Series) -> pandas.core.series.Series>,
 <function texthero.preprocessing.lowercase(input: pandas.core.series.Series) -> pandas.core.series.Series>,
 <function texthero.preprocessing.remove_digits(input: pandas.core.series.Series, only_blocks=True) -> pandas.core.series.Series>,
 <function texthero.preprocessing.remove_punctuation(input: pandas.core.series.Series) -> pandas.core.series.Series>,
 <function texthero.preprocessing.remove_diacritics(input: pandas.core.series.Series) -> pandas.core.series.Series>,
 <function texthero.preprocessing.remove_stopwords(input: pandas.core.series.Series, stopwords: Union[Set[str], NoneType] = None, remove_str_numbers=False) -> pandas.core.series.Series>,
 <function texthero.preprocessing.remove_whitespace(input: pandas.core.series.Series) -> pandas.core.series.Series>]

In [503]:
selected_functions_indexes = [0, 1, 2, 5, 6]
pipeline = [pipeline[i] for i in selected_functions_indexes]
df['text'] = hero.preprocessing.clean(df['text'])
df['text']

0      negocio de reconhecimento facialnao funciona n...
1      um lixo tentativas de econhecimento facial sem...
2      horrivel pior que fbi se fosse pra receberem a...
3           meio dificil fazer sem oculos mais deu certo
4         nao serve pra nada nao da para acessar pessimo
                             ...                        
925    como eu faco pra entrar na minha conta enem pe...
926    sim facilita muito em varios servicos e mais i...
927    nao consigo acessar meu auxilio como faco pra ...
928                                  seguro hiper seguro
929    otimo porem e bom implementar novas funcionali...
Name: text, Length: 930, dtype: object

### Stopwords

Removendo stopwords em português com o corpus em portugês do NTLK

In [504]:
from nltk.corpus import stopwords
stopwords = stopwords.words('portuguese')
df['text'] = hero.remove_stopwords(df['text'], stopwords=stopwords)
df['text'][0]

'negocio  reconhecimento facialnao funciona nao  pra ficar dia todo nisso nao arrumem   favor'

### Stemização

In [505]:
df['text'] = hero.stem(df['text'], language='portuguese')
df['text']
df['text'][0]

'negoci reconhec facialna funcion nao pra fic dia tod niss nao arrum favor'

### Tokenização

In [506]:
df['text'] = hero.tokenize(df['text'])
df.head()

,text,is_usability,is_classified
0,"[negoci, reconhec, facialna, funcion, nao, pra...",True,True
1,"[lix, tentat, econhec, facial, exit, dur, depe...",False,True
2,"[horrivel, pior, fbi, pra, receb, algo, gent, ...",True,True
3,"[mei, dificil, faz, ocul, deu, cert]",True,True
4,"[nao, serv, pra, nad, nao, acess, pessim]",True,True


In [507]:
df['class_name'] = df.apply(lambda row: 'usability' if row['is_usability'] else 'not_usability', axis=1)

In [508]:
df = df.drop('is_usability', 1)
df = df.to_dict('records')
df[0]

{'text': ['negoci',
  'reconhec',
  'facialna',
  'funcion',
  'nao',
  'pra',
  'fic',
  'dia',
  'tod',
  'niss',
  'nao',
  'arrum',
  'favor'],
 'is_classified': True,
 'class_name': 'usability'}

## NLP

Nessa parte, iremos treinar e utilizar o modelo

Função para aprendizado dos dados: 

In [518]:
def learning(training_data):
    corpus_words = {}
    for data in training_data: 
        class_name = data['class_name']
        frase = data['text']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

Função para calcular scores

In [519]:
def classificate(corpus, sentence):
    def calculate_class_score(corpus_words, sentence, class_name):
        score = 0 
        for word in sentence:
            if word in corpus_words[class_name]:
                score += corpus_words[class_name][word]
        return score
    classifications = []
    for class_name in corpus.keys():
        classifications.append({'class_name': class_name, 'score': calculate_class_score(corpus, sentence, class_name)})    
    return 

Função para normalizar os scores

In [520]:
def normalize_scores(classification):
    total_score = sum(score['score'] for score in classification['scores'])
    for score in classification['scores']:
        score['score'] = score['score']/total_score
    return classification

Dividindo o dataset dos classificados em teste e treino

In [521]:
import random

classified_df = []
unclassified_df = []

for data in df:
    classified_df.append(data) if data['is_classified'] else unclassified_df.append(data)

train_dataset = []
test_dataset = []
    
for i in classified_df:
    r = random.uniform(0,1)
    if r <= 0.1:
        test_dataset.append(i)
    else:
        train_dataset.append(i)

print(f'Total test cases {len(test_dataset)}')
print(f'Total train cases {len(train_dataset)}')

Total test cases 39
Total train cases 343


In [522]:
corpus = learning(classified_df)
print(f'qtd usability keys {len(corpus["usability"].keys())}')
print(f'qtd not_usability keys {len(corpus["not_usability"].keys())}')

qtd usability keys 765
qtd not_usability keys 568


In [523]:
classifications = [{'test_data': test_data, 'scores': get_scores(corpus, test_data['text'])} for test_data in test_dataset]

In [524]:
classifications = [normalize_scores(classification) for classification in classifications]
classifications[0]

{'test_data': {'text': ['pessim', 'pessim', 'pessim', 'nad', 'nel', 'funcion'],
  'is_classified': True,
  'class_name': 'not_usability'},
 'scores': [{'class_name': 'usability', 'score': 0.45302013422818793},
  {'class_name': 'not_usability', 'score': 0.5469798657718121}]}

In [525]:
for i, classification in enumerate(classifications):
    message = ""
    message = message + f'Test {i} - Has:'
    for score in classification['scores']:
        message = message + " " + score['class_name'] + " - " + str(score['score'])  + " and "
    message = message + "original result is: " + classification['test_data']['class_name']
    print(message)

Test 0 - Has: usability - 0.45302013422818793 and  not_usability - 0.5469798657718121 and original result is: not_usability
Test 1 - Has: usability - 0.6361788617886179 and  not_usability - 0.3638211382113821 and original result is: usability
Test 2 - Has: usability - 0.6589147286821705 and  not_usability - 0.34108527131782945 and original result is: usability
Test 3 - Has: usability - 0.6176470588235294 and  not_usability - 0.38235294117647056 and original result is: not_usability
Test 4 - Has: usability - 0.6339227242960053 and  not_usability - 0.3660772757039948 and original result is: not_usability
Test 5 - Has: usability - 0.5555555555555556 and  not_usability - 0.4444444444444444 and original result is: not_usability
Test 6 - Has: usability - 0.7076648841354723 and  not_usability - 0.29233511586452765 and original result is: usability
Test 7 - Has: usability - 0.0 and  not_usability - 1.0 and original result is: not_usability
Test 8 - Has: usability - 0.6878612716763006 and  not_

In [526]:
right_answers = 0
for i, classification in enumerate(classifications):
    higher_score = max(classification['scores'], key=lambda x:x['score'])
    if classification['test_data']['class_name'] == higher_score['class_name']:
        right_answers += 1

print(f'Final accuracy is: {right_answers/len(classifications)}')

Final accuracy is: 0.5384615384615384
